In [458]:
from __future__ import print_function, division
import sys
from bs4 import BeautifulSoup
import requests

requests.__path__

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
import random

import pandas as pd
import pickle

In [326]:
chromedriver = "/Users/georgevarelas/Downloads/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

In [507]:
def get_historical_prices(link):
    url = "https://www.transfermarkt.com" + link.replace('profil', 'marktwertverlauf')
    driver = webdriver.Chrome(chromedriver)
    driver.get(url)
    
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    script = soup.find_all('script')
    
    data = (str(script[len(script)-1]))
    data = data.split("'series':[{")[1].split("\'y")
    
    age = []
    value = []
    club = []
    date = []

    for i in range(1, len(data)):
        value.append(data[i].split(':')[1].split(',')[0])
        age.append(data[i].split(':')[3].split(',')[0])
        club.append(data[i].split(':')[2].split(',')[0].replace("'", "").replace("\\x20", " ").replace("\\x2D", " "))

        day = data[i].split(':')[5].split(',')[:-1]
        date.append(day[0][1:4] + " " + day[0][-2:] + ", "+ day[1][-5:-1])
    
    

    
    find_agents = soup.find_all("div", {"class":"row"})
    try:
        agent = str(find_agents).split('<span class="dataItem">Agent:</span>')[1]
        agent_name = agent.split('title="')[1].split('"')[0]
    except IndexError:
        agent_name = 'NAN'
    
    
    
    historicals = list(zip(*[date, value, age, club])) + [agent_name]
    
    driver.close()
    
    return historicals

In [431]:
get_historical_prices("/raheem-sterling/profil/spieler/134425")

[('Sep 11, 2012', '2500000', '17', 'Liverpool FC'),
 ('Jan 28, 2013', '11500000', '18', 'Liverpool FC'),
 ('Jul 02, 2013', '12500000', '18', 'Liverpool FC'),
 ('Jan 15, 2014', '13000000', '19', 'Liverpool FC'),
 ('Aug 12, 2014', '20000000', '19', 'Liverpool FC'),
 ('Feb 12, 2015', '30000000', '20', 'Liverpool FC'),
 ('Jul 01, 2015', '40000000', '20', 'Liverpool FC'),
 ('Oct 14, 2015', '50000000', '20', 'Manchester City'),
 ('Feb 09, 2016', '50000000', '21', 'Manchester City'),
 ('Jun 01, 2016', '45000000', '21', 'Manchester City'),
 ('Aug 01, 2016', '45000000', '21', 'Manchester City'),
 ('Feb 20, 2017', '45000000', '22', 'Manchester City'),
 ('Jun 28, 2017', '45000000', '22', 'Manchester City'),
 ('Oct 23, 2017', '50000000', '22', 'Manchester City'),
 ('Jan 02, 2018', '80000000', '23', 'Manchester City'),
 ('May 28, 2018', '90000000', '23', 'Manchester City'),
 ('Dec 19, 2018', '120000000', '24', 'Manchester City'),
 'Colossal Sports Management']

In [246]:
players = pd.read_pickle('/Users/georgevarelas/Luther/Players_Transfermarkt.pkl')

In [432]:
players.head(3)

,Players,Profile_Link,Club,Values,Old Values,Position,Age,Matches,Goals,Own Goals,Assists,Yellow Cards,Second Yellow,Red Cards,Sub In,Sub Out
0,Kylian Mbappé,/kylian-mbappe/profil/spieler/342229,Paris Saint-Germain,180.0,180.0,Centre-Forward,20,52,43,0,20,8,0,2,8,7
1,Neymar,/neymar/profil/spieler/68290,Paris Saint-Germain,162.0,162.0,Left Winger,27,34,26,0,19,9,0,0,2,9
2,Lionel Messi,/lionel-messi/profil/spieler/28003,FC Barcelona,144.0,162.0,Right Winger,31,51,51,0,22,3,0,0,7,1


In [371]:
d = {}

In [505]:
not_appended = []

for i in (players['Profile_Link']):
    count = 0
    if count % 25 == 0:
        f = open("players_dictionary.pkl","wb")
        pickle.dump(d,f)
        f.close()
    try:
        d[i] = get_historical_prices(i)
        count += 1
    except IndexError:
        print(i)
        not_appended.append(i)
        continue
        
print(len(not_appended))

In [515]:
second_attempt_failed = []

for i in not_appended:
    count = 0
    if count % 25 == 0:
        f = open("players_dictionary.pkl","wb")
        pickle.dump(d,f)
        f.close()
    try:
        d[i] = get_historical_prices(i)
        count += 1
    except IndexError:
        print(i)
        second_attempt_failed.append(i)
        continue

In [518]:
third_attempt_failed = []

for i in second_attempt_failed:
    count = 0
    if count % 25 == 0:
        f = open("players_dictionary.pkl","wb")
        pickle.dump(d,f)
        f.close()
    try:
        d[i] = get_historical_prices(i)
        count += 1
    except IndexError:
        print(i)
        third_attempt_failed.append(i)
        continue

In [523]:
fourth_attempt_failed = []

for i in third_attempt_failed:
    count = 0
    if count % 25 == 0:
        f = open("players_dictionary.pkl","wb")
        pickle.dump(d,f)
        f.close()
    try:
        d[i] = get_historical_prices(i)
        count += 1
    except IndexError:
        print(i)
        fourth_attempt_failed.append(i)
        continue

In [527]:
f = open("players_dictionary.pkl","wb")
pickle.dump(d,f)
f.close()

In [532]:
players['Historical_Values'] = players['Profile_Link'].map(d)

In [546]:
players['Agent'] = players['Historical_Values'].apply(lambda row: row[len(row) - 1])

In [548]:
players['Historical_Values'] = players['Historical_Values'].apply(lambda row: row[:len(row) - 1])

In [552]:
players.to_pickle("Transfermarkt_Historicals.pkl")